# Test Notebook Graph

In [33]:
import pandas as pd
from py2neo import Graph
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

# host + port
host = 'http://localhost:7474'

# select database name
db_name = 'modelparallel'

# credentials for API
user = 'neo4j'
passwd = '1234'

# connect to database
call_graph = Graph(host, auth=(user, passwd), name=db_name)

## Test functions here

### get molecules with more than 10 outgoing transformations and sort by weight

In [47]:
df_transformations = call_graph.run("""
MATCH (m:Molecule)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
WITH m, count(t) as count_HTI
MATCH (m2:Molecule)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
WHERE m.formula_string = m2.formula_string
AND m.sample_id = m2.sample_id
AND count_HTI >= 10
RETURN m2.formula_string as formula_string, m2.sample_id as sample_id, count_HTI as outgoing_transformations, collect({transformation_unit: t.transformation_unit, weight_combined: t.combined_weight, weight_connected: t.connected_weight}) as transformation_details
""").to_data_frame()

In [48]:
df_transformations = df_transformations.sort_values(by=['outgoing_transformations'], ascending=False).reset_index(drop=True)

In [50]:
full_sorted_list = []
for row in df_transformations.itertuples():
    sorted_list = sorted(row.transformation_details, key=lambda d: d['weight_connected'], reverse=True)
    #print(sorted_list)
    sorted_transformation_list = []
    for ele in sorted_list:
        sorted_transformation_list.append(ele['transformation_unit'])
    full_sorted_list.append(sorted_transformation_list)
df_transformations['sorted_transformations_connected_weight'] = full_sorted_list

In [55]:
df_transformations.to_csv('molecules_outgoing_transformations.csv', sep=',', encoding='utf-8', index=False)

### molecules with highest intensity variation and their outgoing transformations

#### top 100 most intensity gain

In [11]:
df_most_gain = call_graph.run("""
MATCH (m:Molecule)-[s:SAME_AS]->(m2:Molecule)
RETURN m.formula_string as formula_string, m.sample_id as sample_id_tp1, m.peak_relint_tic as relative_intensity_tp1,
m2.sample_id as sample_id_tp2, m2.peak_relint_tic as relative_intensity_tp2, s.intensity_trend as intensity_gain
ORDER BY s.intensity_trend DESC LIMIT 100
""").to_data_frame()

In [38]:
df_most_gain

,formula_string,sample_id_tp1,relative_intensity_tp1,sample_id_tp2,relative_intensity_tp2,intensity_gain
0,C18 H35 N1 O7,7857,0.000042,7863,0.000199,4.786
1,C14 H22 O1,7851,0.000085,7857,0.000395,4.661
2,C22 H39 N1 O3,7827,0.000027,7833,0.000122,4.549
3,C18 H36 O2,7839,0.002149,7845,0.008336,3.878
4,C10 H20 O2,7863,0.000251,7868,0.000941,3.757
...,...,...,...,...,...,...
95,C10 H6 O9,7809,0.000028,7815,0.000061,2.179
96,C16 H30 O5,7809,0.000092,7815,0.000199,2.169
97,C10 H18 O3,7809,0.000098,7815,0.000213,2.166
98,C8 H14 O3,7839,0.000038,7845,0.000081,2.153


In [62]:
df_most_gain.to_csv('molecules_most_intensity_gain.csv', sep=',', encoding='utf-8', index=False)

#### most intensity loss 

In [68]:
df_most_loss = call_graph.run("""
MATCH (m:Molecule)-[s:SAME_AS]->(m2:Molecule)
OPTIONAL MATCH (m)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
RETURN m.formula_string as formula_string, m.sample_id as sample_id_tp1, m.peak_relint_tic as relative_intensity_tp1,
m2.sample_id as sample_id_tp2, m2.peak_relint_tic as relative_intensity_tp2, s.intensity_trend as intensity_gain,
collect(t.transformation_unit) as outgoing_transformations
ORDER BY s.intensity_trend ASC LIMIT 100
""").to_data_frame()

In [69]:
df_most_loss

,formula_string,sample_id_tp1,relative_intensity_tp1,sample_id_tp2,relative_intensity_tp2,intensity_gain,outgoing_transformations
0,C22 H41 N1 O3,7815,0.000433,7821,0.000037,0.085,[]
1,C22 H43 N1 O1,7815,0.001337,7821,0.000125,0.093,[]
2,C22 H43 N1 O3,7815,0.000598,7821,0.000057,0.095,[]
3,C22 H41 N1 O2,7815,0.000822,7821,0.000083,0.101,[]
4,C22 H39 N1 O3,7815,0.000489,7821,0.000051,0.105,[]
...,...,...,...,...,...,...,...
95,C10 H20 O2,7857,0.000587,7863,0.000251,0.427,"[-C1 -H2 O1, -H2, -H2 O1, -H4 O2]"
96,C16 H31 N1 O5,7868,0.000347,7874,0.000148,0.427,[]
97,C13 H25 N1 O4,7851,0.000044,7857,0.000019,0.429,"[-C1 -H2 O1, O1, -H2 O1]"
98,C18 H35 N1 O7,7868,0.000373,7874,0.000160,0.429,[-H1 O1 -N1]


In [70]:
df_most_loss.to_csv('molecules_most_intensity_loss.csv', sep=',', encoding='utf-8', index=False)